In [1]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
import matplotlib.pyplot as plt

In [2]:
input_dim=784 #Mnist: 28x28 images -> input dimension = 784
hidden_dim=128 #width of the hidden layer
output_dim=10 #10 classes (the digits in mnist)

In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)
                
    def forward(self, x):
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

Slight modification to ```ff_main.py``` file:
- the ```train``` method has an added param ```reg```, that trains
  the model using $L_{reg}$ regularization. For example, when using ```reg=1```,
  the model is trained using $L_1$ reguralization and when using ```reg=2```,
  $L_2$ reguralization is used.

In [4]:
def train(model, device, train_loader, optimizer, epoch, reg=2):
    model.train()

    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)

        # Adding L_reg loss
        l_reg = 0.0
        alpha=0.001
        for param in model.parameters():
            l_reg += alpha*torch.norm(param, reg) ** reg
        loss += l_reg

        loss.backward()
        optimizer.step()

        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

Slight modification to ```ff_main.py``` file: the ```test``` method returns the test rate.

In [5]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    return correct / len(test_loader.dataset)

In [6]:
batch_size = 100
test_batch_size= 1000
epochs = 14
lr = 0.1
gamma = 0.7
seed = 1
log_interval = 60
save_model=False

In [7]:
torch.manual_seed(seed)

device = "cpu" # "cuda"

train_kwargs = {'batch_size': batch_size}
test_kwargs = {'batch_size': test_batch_size}
if device == "cuda":
    cuda_kwargs = {'num_workers': 1,
                   'pin_memory': True,
                   'shuffle': True}

In [8]:
transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
    ])
dataset1 = datasets.MNIST('../data', train=True, download=False,
                   transform=transform)
dataset2 = datasets.MNIST('../data', train=False, download=False,
                   transform=transform)
train_loader = torch.utils.data.DataLoader(dataset1,**train_kwargs)
test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

In [9]:
model = Net().to(device)
optimizer = optim.Adadelta(model.parameters(), lr=lr)

In [ ]:
accs = {}
epochs = 14

for reg in range(1,3):
    scheduler = StepLR(optimizer, step_size=1, gamma=gamma)
    acc = []
    print(f"Training using the L'{reg}' reguralization")
    for epoch in range(1, epochs + 1):
        train(model, device, train_loader, optimizer, epoch, reg)
        a = test(model, device, test_loader)
        acc.append(a)
        scheduler.step()
    accs[reg] = acc
    print()

Training using the L'1' reguralization
Train Epoch: 1 [0/60000 (0%)]	Loss: 4.146940
Train Epoch: 1 [6000/60000 (10%)]	Loss: 2.064221
Train Epoch: 1 [12000/60000 (20%)]	Loss: 1.698361
Train Epoch: 1 [18000/60000 (30%)]	Loss: 1.411469
Train Epoch: 1 [24000/60000 (40%)]	Loss: 1.176971
Train Epoch: 1 [30000/60000 (50%)]	Loss: 1.137388
Train Epoch: 1 [36000/60000 (60%)]	Loss: 1.057780
Train Epoch: 1 [42000/60000 (70%)]	Loss: 0.874290
Train Epoch: 1 [48000/60000 (80%)]	Loss: 0.783380
Train Epoch: 1 [54000/60000 (90%)]	Loss: 0.919809

Test set: Average loss: 0.3730, Accuracy: 9007/10000 (90%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.742363


In [ ]:
import numpy as np
for i, acc in accs.items():
    plt.plot(1 - np.array(acc), label=f"L{i}")

plt.xlabel("Epoch")
plt.ylabel("Error rate")
plt.legend()
plt.show()

In [ ]:
c1=0
threshold=1e-4
for (i,p) in enumerate(model.parameters()):
#in this case i==0 and i==2 refer to weight matrices
    if i==0 or i==2:
        c1+=sum(sum(torch.abs(p.data)<threshold)).item()
c1